In [1]:
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import functional as F
import torch
import torch.nn as nn
from PIL import Image,ImageOps
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from torchvision.utils import save_image, make_grid
import random
import os

dataset_path = '~/datasets'

cuda = True
DEVICE = torch.device("cuda" if cuda else "cpu")

batch_size = 50
x_dim  = 784
hidden_dim = 400
latent_dim = 200
lr = 1e-3
epochs = 30

In [2]:
c = 0.3
p = 0.5
batch_size = 30
class MNISTPairDataset(Dataset):
    def __init__(self, c, p, mnist_dataset):
        self.mnist_dataset = mnist_dataset
        self.c = c
        self.p = p
        # 创建一个存储每个数字图片索引的字典
        self.label_to_indices = {i: [] for i in range(10)}
        for idx, (_, label) in enumerate(self.mnist_dataset):
            self.label_to_indices[label].append(idx)
        self.target_index = {0: 422, 1: 726, 2: 400, 3: 632, 4: 344, 5: 242, 6: 217, 7: 667, 8: 342, 9: 508}

    def __len__(self):
        return len(self.mnist_dataset)
    def convert(self,gray_image):
      gray_image = gray_image.numpy()
      gray_image = gray_image[0]
      rgb_image = np.stack([gray_image] * 3, axis=-1)

      return rgb_image
    def __getitem__(self, idx):
        input_img, input_label = self.mnist_dataset[idx]

        angle = random.uniform(-15, 15)
        input_img = F.rotate(input_img, angle, fill=0)
        #random_num = self.c + (1 - self.c) * torch.rand(1)
        # random_num = torch.rand(1)
        # if random_num < self.p:
        #   #input_img = input_img * random_num
        #   input_img = input_img * self.c
        output_label = (input_label + 1) % 10
        output_idx = 0
        output_img = self.mnist_dataset[self.label_to_indices[output_label][output_idx]][0]

        return input_img, output_img

mnist_transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3, 1, 1))
])

kwargs = {'num_workers': 1, 'pin_memory': True}

train_dataset = MNIST(dataset_path, transform=mnist_transform, train=True, download=True)
test_dataset  = MNIST(dataset_path, transform=mnist_transform, train=False, download=True)



train_dataset = MNISTPairDataset(c,p,train_dataset)
test_dataset = MNISTPairDataset(c,p,test_dataset)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False, **kwargs)
test_loader  = DataLoader(dataset=test_dataset,  batch_size=batch_size, shuffle=False, **kwargs)


Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 9.91M/9.91M [00:00<00:00, 52.4MB/s]


Extracting /root/datasets/MNIST/raw/train-images-idx3-ubyte.gz to /root/datasets/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 28.9k/28.9k [00:00<00:00, 1.50MB/s]


Extracting /root/datasets/MNIST/raw/train-labels-idx1-ubyte.gz to /root/datasets/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 1.65M/1.65M [00:00<00:00, 12.9MB/s]


Extracting /root/datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/datasets/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 4.54k/4.54k [00:00<00:00, 4.43MB/s]

Extracting /root/datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/datasets/MNIST/raw



Function

In [3]:
mse_loss = torch.nn.MSELoss()

def calc_mean_std(feat, eps=1e-5):
    # eps is a small value added to the variance to avoid divide-by-zero.
    size = feat.size()
    assert (len(size) == 4)
    N, C = size[:2]
    feat_var = feat.view(N, C, -1).var(dim=2) + eps
    feat_std = feat_var.sqrt().view(N, C, 1, 1)
    feat_mean = feat.view(N, C, -1).mean(dim=2).view(N, C, 1, 1)
    return feat_mean, feat_std


def adain(content_feat, style_feat):
    assert (content_feat.size()[:2] == style_feat.size()[:2])
    size = content_feat.size()
    style_mean, style_std = calc_mean_std(style_feat)
    content_mean, content_std = calc_mean_std(content_feat)

    normalized_feat = (content_feat - content_mean.expand(
        size)) / content_std.expand(size)
    return normalized_feat * style_std.expand(size) + style_mean.expand(size)


def _calc_feat_flatten_mean_std(feat):
    # takes 3D feat (C, H, W), return mean and std of array within channels
    assert (feat.size()[0] == 3)
    assert (isinstance(feat, torch.FloatTensor))
    feat_flatten = feat.view(3, -1)
    mean = feat_flatten.mean(dim=-1, keepdim=True)
    std = feat_flatten.std(dim=-1, keepdim=True)
    return feat_flatten, mean, std


def _mat_sqrt(x):
    U, D, V = torch.svd(x)
    return torch.mm(torch.mm(U, D.pow(0.5).diag()), V.t())


def coral(source, target):
    # assume both source and target are 3D array (C, H, W)
    # Note: flatten -> f

    source_f, source_f_mean, source_f_std = _calc_feat_flatten_mean_std(source)
    source_f_norm = (source_f - source_f_mean.expand_as(
        source_f)) / source_f_std.expand_as(source_f)
    source_f_cov_eye = \
        torch.mm(source_f_norm, source_f_norm.t()) + torch.eye(3)

    target_f, target_f_mean, target_f_std = _calc_feat_flatten_mean_std(target)
    target_f_norm = (target_f - target_f_mean.expand_as(
        target_f)) / target_f_std.expand_as(target_f)
    target_f_cov_eye = \
        torch.mm(target_f_norm, target_f_norm.t()) + torch.eye(3)

    source_f_norm_transfer = torch.mm(
        _mat_sqrt(target_f_cov_eye),
        torch.mm(torch.inverse(_mat_sqrt(source_f_cov_eye)),
                 source_f_norm)
    )

    source_f_transfer = source_f_norm_transfer * \
                        target_f_std.expand_as(source_f_norm) + \
                        target_f_mean.expand_as(source_f_norm)

    return source_f_transfer.view(source.size())

Net framework

In [4]:
Decoder = nn.Sequential(
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 256, (3, 3)),
    nn.ReLU(),
    nn.Upsample(scale_factor=2, mode='nearest'),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 128, (3, 3)),
    nn.ReLU(),
    nn.Upsample(scale_factor=2, mode='nearest'),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(128, 128, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(128, 64, (3, 3)),
    nn.ReLU(),
    nn.Upsample(scale_factor=2, mode='nearest'),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(64, 64, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(64, 3, (3, 3)),
)

vgg = nn.Sequential(
    nn.Conv2d(3, 3, (1, 1)),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(3, 64, (3, 3)),
    nn.ReLU(),  # relu1-1
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(64, 64, (3, 3)),
    nn.ReLU(),  # relu1-2
    nn.MaxPool2d((2, 2), (2, 2), (0, 0), ceil_mode=True),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(64, 128, (3, 3)),
    nn.ReLU(),  # relu2-1
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(128, 128, (3, 3)),
    nn.ReLU(),  # relu2-2
    nn.MaxPool2d((2, 2), (2, 2), (0, 0), ceil_mode=True),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(128, 256, (3, 3)),
    nn.ReLU(),  # relu3-1
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),  # relu3-2
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),  # relu3-3
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),  # relu3-4
    nn.MaxPool2d((2, 2), (2, 2), (0, 0), ceil_mode=True),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 512, (3, 3)),
    nn.ReLU(),  # relu4-1, this is the last layer used
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(),  # relu4-2
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(),  # relu4-3
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(),  # relu4-4
    nn.MaxPool2d((2, 2), (2, 2), (0, 0), ceil_mode=True),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(),  # relu5-1
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(),  # relu5-2
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(),  # relu5-3
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU()  # relu5-4
)

# class VQVAEcodebook(nn.Module):
#     def __init__(self, num_embeddings, embedding_dim, commitment_cost=0.25):
#         super(VQVAEcodebook, self).__init__()

#         self.vq_embedding = nn.Embedding(num_embeddings, embedding_dim)
#         self.vq_embedding.weight.data.uniform_(-1.0 / num_embeddings, 1.0 / num_embeddings)
#         # self.conv1 = nn.Conv2d(512,embedding_dim,1)
#         # self.conv2 = nn.Conv2d(embedding_dim,512,1)

#     def forward(self, ze):
#         N, C, H, W = ze.shape
#         embedding = self.vq_embedding.weight.data
#         K, _ = embedding.shape

#         embedding_broadcast = embedding.reshape(1, K, C, 1, 1)
#         ze_broadcast = ze.reshape(N, 1, C, H, W)

#         distance = torch.sum((embedding_broadcast - ze_broadcast) ** 2, 2)  # (N,K,H,W)
#         nearest_neghbor = torch.argmin(distance, 1)  # (N,H,W)

#         # zq (N, C, H, W) : (N, H, W, C) -> (N, C, H, W)
#         zq = self.vq_embedding(nearest_neghbor).permute(0, 3, 1, 2)

#         codebook_output = ze + (zq - ze).detach()

#         return ze, zq, codebook_output


class ResidualBlock(nn.Module):

    def __init__(self, dim):
        super(ResidualBlock, self).__init__()

        self.res_block = nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(dim, dim, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(dim, dim, 1)
        )

    def forward(self, x):
        x = x + self.res_block(x)
        return torch.relu(x)


class Net(nn.Module):
    def __init__(self, encoder, decoder, n_embedding, dim):
        super(Net, self).__init__()
        enc_layers = list(encoder.children())
        self.enc_1 = nn.Sequential(*enc_layers[:4])  # input -> relu1_1
        self.enc_2 = nn.Sequential(*enc_layers[4:11])  # relu1_1 -> relu2_1
        self.enc_3 = nn.Sequential(*enc_layers[11:18])  # relu2_1 -> relu3_1
        self.enc_4 = nn.Sequential(*enc_layers[18:31])  # relu3_1 -> relu4_1
        self.decoder = decoder
        #self.codebook = codebook
        self.mse_loss = torch.nn.MSELoss()

        # fix the encoder and decoder
        for name in ['enc_1', 'enc_2', 'enc_3', 'enc_4']:
            for param in getattr(self, name).parameters():
                param.requires_grad = False
        for param in self.decoder.parameters():
            param.requires_grad = False
        #codebook
        self.vq_embedding = nn.Embedding(n_embedding, dim)
        self.vq_embedding.weight.data.uniform_(-1.0 / n_embedding, 1.0/n_embedding)

        self.encoderx = nn.Sequential(
            nn.Conv2d(512, dim, 3, 1, 1),
            nn.ReLU(),#relu_1
            nn.Conv2d(dim, dim, 3, 1, 1),
            nn.ReLU(),#relu_2
            nn.Conv2d(dim, dim, 3, 1, 1),
            ResidualBlock(dim),
            ResidualBlock(dim)
        )

        self.decoderx = nn.Sequential(
            nn.Conv2d(dim, dim, 3, 1, 1),
            ResidualBlock(dim),
            ResidualBlock(dim),
            nn.ConvTranspose2d(dim, dim, 3, 1, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(dim, 512, 3, 1, 1)
        )
    # extract relu1_1, relu2_1, relu3_1, relu4_1 from input image
    def encode_with_intermediate(self, input):
        results = [input]
        for i in range(4):
            func = getattr(self, 'enc_{:d}'.format(i + 1))
            results.append(func(results[-1]))
        return results[1:]

    # extract relu4_1 from input image
    def encode(self, input):
        for i in range(4):
            input = getattr(self, 'enc_{:d}'.format(i + 1))(input)
        return input

    def calc_content_loss(self, input, target):
        assert (input.size() == target.size())
        assert (target.requires_grad is False)
        return self.mse_loss(input, target)

    def calc_style_loss(self, input, target):
        assert (input.size() == target.size())
        assert (target.requires_grad is False)
        input_mean, input_std = calc_mean_std(input)
        target_mean, target_std = calc_mean_std(target)
        return self.mse_loss(input_mean, target_mean) + \
               self.mse_loss(input_std, target_std)

    def forward(self, content, style, label, alpha):

        x = self.encode(content)
        mean,std = calc_mean_std(x)
        x = (x - mean) / std

        ze = self.encoderx(x)
        embedding = self.vq_embedding.weight.data
        N, C, H, W = ze.shape
        K, _ = embedding.shape

        embedding_broadcast = embedding.reshape(1, K, C, 1, 1)
        ze_broadcast = ze.reshape(N, 1, C, H, W)

        distance = torch.sum((embedding_broadcast - ze_broadcast) ** 2, 2)  # (N,K,H,W)
        nearest_neghbor = torch.argmin(distance, 1)  # (N,H,W)


        # zq (N, C, H, W) : (N, H, W, C) -> (N, C, H, W)
        zq = self.vq_embedding(nearest_neghbor).permute(0, 3, 1, 2)

        t = ze + (zq - ze).detach()
        t = self.decoderx(t)

        mean_t,std_t = calc_mean_std(t)
        gt = (t - mean_t) / std_t
        gt = gt * std + mean
        gt = alpha * gt + (1 - alpha) * t
        gt = self.decoder(gt)
        label = self.encode(label)
        gt_feature = self.encode(gt)

        loss_r = mse_loss(label,gt_feature)
        mean_label,std_label = calc_mean_std(label)
        label = (label - mean_label) / std_label

        # mean_t,std_t = calc_mean_std(t)
        # t = (t - mean_t) / std_t

        loss_rec = mse_loss(label,t)
        return ze, zq, loss_rec, loss_r, gt
##10_22 checkpoint

Load Model

In [5]:
decoder = Decoder
encoder = vgg
model = Net(encoder, decoder, 200,100)
state_dict = torch.load("/content/final_model4_200_100.pth")
model.load_state_dict(state_dict)
model.train()
model.to(DEVICE)


<ipython-input-5-e805ffc750d0>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("/content/final_model4_200_100.pth")


Net(
  (enc_1): Sequential(
    (0): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
    (1): ReflectionPad2d((1, 1, 1, 1))
    (2): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
  )
  (enc_2): Sequential(
    (0): ReflectionPad2d((1, 1, 1, 1))
    (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0), dilation=1, ceil_mode=True)
    (4): ReflectionPad2d((1, 1, 1, 1))
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (6): ReLU()
  )
  (enc_3): Sequential(
    (0): ReflectionPad2d((1, 1, 1, 1))
    (1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0), dilation=1, ceil_mode=True)
    (4): ReflectionPad2d((1, 1, 1, 1))
    (5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
    (6): ReLU()
  )
  (enc_4): Sequential(
    (0): ReflectionPad2d((1, 1, 1, 1))
    (1): Conv2d(256, 256,

Train

In [43]:
#optimizer = torch.optim.Adam(model.parameters(),lr=1e-4,weight_decay=1e-4)
optimizer = torch.optim.SGD(model.parameters(),lr = 1e-4 ,momentum=0.9)
epochs = 5
alpha=1
beta= 1
gamma = 0.25
minloss = 1e9
checkpoint = "vqvae_checkpoints.pth"
print("start vqvae train...")

losslist = []
for epo in range(epochs):
    overall_loss = 0
    overall_zq = 0
    overall_z = 0
    overall_r = 0
    for batch_idx, (x, y) in enumerate(train_loader):
        x = x.to(DEVICE)  # N1HW
        y = y.to(DEVICE)

        ze, zq, loss_rec, loss_r ,x_hat = model(x,x,y,1)
        loss_ze = mse_loss(zq.detach(), ze)
        loss_zq = mse_loss(zq, ze.detach())

        loss = loss_zq + 0.25 * loss_ze + loss_rec + loss_r

        overall_loss += loss
        overall_r += loss_rec + loss_r
        overall_zq += loss_zq
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #break
    losslist.append(overall_loss.item())
    print(f"epoch:{epo}, loss_overall:{overall_loss.item():.6f}")
    print(f"epoch:{epo}, loss_rec:  {overall_r.item():.6f}")
    print(f"epoch:{epo}, loss_zq:   {overall_zq.item():.6f}")
    #print(f"epoch:{epo}, loss_z:   {overall_z.item():.6f}")
    print()
    #break
    # if (epo+1) % 10 == 0:
    #     torch.save(model.state_dict(), checkpoint)
    if minloss > overall_loss:
        torch.save(model.state_dict(), checkpoint)
        minloss = overall_loss
print("vqvae train finish!!")

start vqvae train...
epoch:0, loss_overall:1553.172119
epoch:0, loss_rec:  844.832581
epoch:0, loss_zq:   566.672424

epoch:1, loss_overall:1523.729370
epoch:1, loss_rec:  831.809143
epoch:1, loss_zq:   553.537354

epoch:2, loss_overall:1522.709595
epoch:2, loss_rec:  829.095764
epoch:2, loss_zq:   554.890259



KeyboardInterrupt: 

Test

In [ ]:

test_model = Net(encoder, decoder, 200, 100)
state_dict = torch.load("/content/model4_200_100.pth")
test_model.load_state_dict(state_dict)
test_model.to(DEVICE)
test_model.eval()
import cv2
import einops
def reconstruct(model, x, device):
    model.to(device)
    model.eval()
    with torch.no_grad():
        _, _,_,_, x_hat = model(x,x,x,0.9)
    print(x_hat.shape)
    n = x.shape[0]
    n1 = 5

    x_cat = torch.concat((x, x_hat), 3)
    x_cat = einops.rearrange(x_cat, '(n1 n2) c h w -> (n1 h) (n2 w) c', n1=n1)
    x_cat = (x_cat.clip(0, 1) * 255).cpu().numpy().astype(np.uint8)
    cv2.imwrite(f'reconstruct_show.jpg', x_cat)

batch_imgs, _ = next(iter(test_loader))
batch_imgs = batch_imgs.to(DEVICE)
reconstruct(test_model, batch_imgs, DEVICE)


<ipython-input-65-76585b797bf3>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("/content/model4_200_100.pth")


torch.Size([30, 3, 256, 256])


In [45]:

import cv2
import einops
def reconstruct(model, x, device):
    model.to(device)
    model.eval()
    with torch.no_grad():
        _,_,_,_,x_hat = model(x,x,x,1)
    print(x_hat.shape)
    n = x.shape[0]
    n1 = 5

    x_cat = torch.concat((x, x_hat), 3)
    x_cat = einops.rearrange(x_cat, '(n1 n2) c h w -> (n1 h) (n2 w) c', n1=n1)
    x_cat = (x_cat.clip(0, 1) * 255).cpu().numpy().astype(np.uint8)
    cv2.imwrite(f'reconstruct_show.jpg', x_cat)

batch_imgs, _ = next(iter(test_loader))
batch_imgs = batch_imgs.to(DEVICE)
reconstruct(model, batch_imgs, DEVICE)


torch.Size([30, 3, 256, 256])


Pre with VGG

In [ ]:
def style_transfer(vgg, decoder, content, style, alpha=1.0,
                   interpolation_weights=None):
    assert (0.0 <= alpha <= 1.0)
    content_f = vgg(content)
    style_f = vgg(style)
    feat = adain(content_f, style_f)
    feat = feat * alpha + content_f * (1 - alpha)
    # mean,std = calc_mean_std(content_f)
    # feat = (content_f - mean) / std
    return decoder(feat)
import cv2
import einops
def vgg_reconstruct(x, device):

    with torch.no_grad():
        x_hat = style_transfer(encoder,decoder,x,x)

    n = x.shape[0]
    n1 = 1
    x_cat = torch.concat((x, x_hat), 3)
    x_cat = einops.rearrange(x_cat, '(n1 n2) c h w -> (n1 h) (n2 w) c', n1=n1)
    x_cat = (x_cat.clip(0, 1) * 255).cpu().numpy().astype(np.uint8)
    cv2.imwrite(f'reconstruct_show.jpg', x_cat)


# batch_imgs, _ = next(iter(test_loader))
# batch_imgs = batch_imgs.to(DEVICE)
# vgg_reconstruct(batch_imgs, DEVICE)
image_path = '/content/2_172.png'  # 替换为你的图片路径
image = Image.open(image_path)
# image = image.convert('L')
# image = ImageOps.invert(image)
# threshold = 20
# image = image.point(lambda p: 255 if p > threshold else 0)
image_preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),  # 转换为张量，且归一化到 [0, 1]
    transforms.Lambda(lambda x: x[[2, 1, 0], :, :])
    #transforms.Lambda(lambda x: x.repeat(3, 1, 1))
])
input_tensor = image_preprocess(image)  # 得到预处理后的张量
input_batch = input_tensor.unsqueeze(0)  # 增加batch维度，变为(1, C, H, W) 适应模型输入
input_batch = input_batch.to(DEVICE)
vgg_reconstruct(input_batch, DEVICE)

In [52]:
def reconstruct(model, x, device):
    model.to(device)
    model.eval()
    with torch.no_grad():
        _,_,_,_,x_hat = model(x,x,x,1)
    print(x_hat.shape)
    n = x.shape[0]
    n1 = 1

    x_cat = torch.concat((x, x_hat), 3)
    x_cat = einops.rearrange(x_cat, '(n1 n2) c h w -> (n1 h) (n2 w) c', n1=n1)
    x_cat = (x_cat.clip(0, 1) * 255).cpu().numpy().astype(np.uint8)
    cv2.imwrite(f'reconstruct_show.jpg', x_cat)

# test_model = VQVAE(1,200,10)
# state_dict = torch.load("/content/vqvae_checkpoints_2024_10_9_epoch5_loss5.pth")
# test_model.load_state_dict(state_dict)
# test_model.to(DEVICE)
# test_model.eval()
image_path = '/content/0_44.png'  # 替换为你的图片路径
image = Image.open(image_path)
# image = image.convert('L')
# image = ImageOps.invert(image)
# threshold = 20
# image = image.point(lambda p: 255 if p > threshold else 0)
image_preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),  # 转换为张量，且归一化到 [0, 1]
    transforms.Lambda(lambda x: x[[2, 1, 0], :, :])
    #transforms.Lambda(lambda x: x.repeat(3, 1, 1))
])
input_tensor = image_preprocess(image)  # 得到预处理后的张量
input_batch = input_tensor.unsqueeze(0)  # 增加batch维度，变为(1, C, H, W) 适应模型输入
input_batch = input_batch.to(DEVICE)
reconstruct(model, input_batch, DEVICE)

torch.Size([1, 3, 256, 256])


In [ ]:
torch.save(model.state_dict(), checkpoint)

In [ ]:
print(minloss)

1205


In [8]:

import cv2
import einops
def reconstruct(x, y, device):
    # model.to(device)
    # model.eval()
    # with torch.no_grad():
    #     _, _,_, _,x_hat = model(x,x,x,0.9)
    #print(x_hat.shape)
    n = x.shape[0]
    n1 = 5

    x_cat = torch.concat((x, y), 3)
    x_cat = einops.rearrange(x_cat, '(n1 n2) c h w -> (n1 h) (n2 w) c', n1=n1)
    x_cat = (x_cat.clip(0, 1) * 255).cpu().numpy().astype(np.uint8)
    cv2.imwrite(f'reconstruct_show.jpg', x_cat)

batch_imgs, _ = next(iter(test_loader))
batch_imgs = batch_imgs.to(DEVICE)
_ = _.to(DEVICE)
reconstruct(batch_imgs,_, DEVICE)


In [ ]:
print(minloss)

tensor(2546.0906, device='cuda:0', grad_fn=<AddBackward0>)


In [6]:
import cv2
import einops
def reconstruct(model, x,y, device,path):
    model.to(device)
    model.eval()
    with torch.no_grad():
        alpha = 1
        _, _, _,_,x_hat = model(x,x,x,alpha)

    n = x.shape[0]
    n1 = 1
    x_cat = torch.concat((torch.concat((x, x_hat), 2),y),2)
    x_cat = einops.rearrange(x_cat, '(n1 n2) c h w -> (n1 h) (n2 w) c', n1=n1)
    x_cat = (x_cat.clip(0, 1) * 255).cpu().numpy().astype(np.uint8)
    cv2.imwrite(path, x_cat)

model.eval()
batch_imgs, _ = next(iter(test_loader))
batch_img = batch_imgs[10].unsqueeze(0)
label_img = _[10].unsqueeze(0)
batch_imgs = batch_imgs.to(DEVICE)
label_img = label_img.to(DEVICE)
batch_img = batch_img.to(DEVICE)
reconstruct(model, batch_img,label_img, DEVICE,'reconstruct_show.jpg')



In [47]:
# model = VQVAE(3,300,10)
# state_dict = torch.load("/content/vqvae_checkpoints_10_22.pth")
# model.load_state_dict(state_dict)
# model.to(DEVICE)
# model.eval()
import cv2
import einops
alpha = 1.0
def reconstruct(model, x, y,device,path):
    # model.to(device)
    # model.eval()
    # with torch.no_grad():
    #     _, _, _,x_hat = model(x,x,x,alpha)

    # n = x.shape[0]
    # n1 = int(n**0.5)
    # x_cat = torch.concat((x, x_hat), 3)
    # x_cat = einops.rearrange(x_cat, '(n1 n2) c h w -> (n1 h) (n2 w) c', n1=n1)
    # x_cat = (x_cat.clip(0, 1) * 255).cpu().numpy().astype(np.uint8)
    # cv2.imwrite(path, x_cat)
    model.to(device)
    model.eval()
    with torch.no_grad():
        alpha = 1
        _, _, _,_,x_hat = model(x,x,x,alpha)

    n = x.shape[0]
    n1 = 1
    x_cat = torch.concat((torch.concat((x, x_hat), 2),y),2)
    x_cat = einops.rearrange(x_cat, '(n1 n2) c h w -> (n1 h) (n2 w) c', n1=n1)
    x_cat = (x_cat.clip(0, 1) * 255).cpu().numpy().astype(np.uint8)
    cv2.imwrite(path, x_cat)
image_path = '/content/8_45.png'  # 替换为你的图片路径
out_put_path = '/content/9_45.png'
image = Image.open(image_path)
# image = image.convert('L')
# image = ImageOps.invert(image)
# threshold = 20
# image = image.point(lambda p: 255 if p > threshold else 0)
image_preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),  # 转换为张量，且归一化到 [0, 1]
    transforms.Lambda(lambda x: x[[2, 1, 0], :, :])
    #transforms.Lambda(lambda x: x.repeat(3, 1, 1))
])
input_tensor = image_preprocess(image)  # 得到预处理后的张量
input_batch = input_tensor.unsqueeze(0)  # 增加batch维度，变为(1, C, H, W) 适应模型输入
input_batch = input_batch.to(DEVICE)
image = Image.open(out_put_path)
out_tensor = image_preprocess(image)
out_batch = out_tensor.unsqueeze(0)
out_batch = out_batch.to(DEVICE)
reconstruct(model, input_batch, out_batch,DEVICE,'test.png')